In [3]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from forcateri.data.dataprovider import DataProvider, SeriesRole
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
from forcateri.data.timeseries import TimeSeries

In [3]:
%load_ext autoreload
%autoreload 2 

In [4]:
# baltbest = BaltBestAggregatedAPIData(
#     name='test', 
#     url="baltbest_url", 
#     local_copy="/home/user/DFKI/forcateri/_data/showcase_data.csv",
#     target = 'q_hca',
#     group_col = 'room_id',
#     time_col = 'datetime',
#     known = 'temperature_outdoor_avg',
#     observed = ['temperature_1_max', 'temperature_2_max','temperature_room_avg'],)
baltbest = BaltBestAggregatedAPIData()

In [5]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [6]:

#baltbest.get_data()


In [7]:
roles = {
    'q_hca': SeriesRole.TARGET, 
    'temperature_outdoor_avg':SeriesRole.KNOWN, 
    'temperature_1_max':SeriesRole.OBSERVED, 
    'temperature_2_max':SeriesRole.OBSERVED,
    'temperature_room_avg':SeriesRole.OBSERVED,}
#['temperature_1_max', 'temperature_2_max','temperature_room_avg']
# roles = {
#     'delta': SeriesRole.TARGET, 
#     'outside_temp':SeriesRole.KNOWN, 
#     'max_temperature_1':SeriesRole.OBSERVED, 
#     'max_temperature_1':SeriesRole.OBSERVED,
#     'room_temperature':SeriesRole.OBSERVED,}

In [8]:
start = pd.Timestamp(2021, 1, 3, 4,tz=0)
end = pd.Timestamp(2021, 1, 8, 4,tz=0)
dataprovider = DataProvider(data_sources=[baltbest], roles=roles)

2025-07-29 16:58:03,672 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:03,672 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:03,685 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:03,686 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:03,698 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:03,699 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:03,711 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:03,712 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:03,725 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:03,725 - INFO - TimeSeries initialized from in

In [9]:
train = dataprovider.get_train_set()
val = dataprovider.get_val_set()
test = dataprovider.get_test_set()

2025-07-29 16:58:05,485 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:05,487 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:05,492 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:05,493 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:05,497 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:05,497 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:05,501 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:05,502 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-29 16:58:05,505 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-29 16:58:05,506 - INFO - TimeSeries initialized from in

In [10]:
train[0].target.data

feature                    q_hca
representation             value
offset time_stamp               
0 days 2019-06-01 00:00:00   0.0
       2019-06-01 01:00:00   0.0
       2019-06-01 02:00:00   0.0
       2019-06-01 03:00:00   0.0
       2019-06-01 04:00:00   0.0
...                          ...
       2020-01-14 10:00:00   0.0
       2020-01-14 11:00:00   0.0
       2020-01-14 12:00:00   0.0
       2020-01-14 13:00:00   0.0
       2020-01-14 14:00:00   0.0

[5463 rows x 1 columns]

In [11]:
num_forecast_steps = 5
#num_monte_carlo_samples = 50 # A sufficient number of samples to approximate the quantiles
dartstcn = DartsTCNModel.load( "/home/user/DFKI/forcateri/_data/dartstcn.pt")
quantiles = [0.1, 0.5, 0.9] # Define the quantiles you want to predict
dartstcn.quantiles = quantiles
#Here prediction is darts timeseries
prediction = dartstcn.predict(
    data=val, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    forecast_horizon=num_forecast_steps,
)

/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:104: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:104: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:104: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:104: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsm

Last points_only:False


/home/user/DFKI/forcateri/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
2025-07-29 17:07:21,645 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,649 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,652 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,670 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,674 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,678 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,683 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-29 17:07:21,686 - INFO - TimeSeries in

In [1]:
import pickle 

with open("/home/user/DFKI/forcateri/_data/historicals_5ts.pkl", "rb") as f:
    data = pickle.load(f)

In [6]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsModelAdapter

prediction_ts_format = [DartsModelAdapter.to_time_series(ts=pred,quantiles=[0.1,0.5,0.9]) for pred in data]

In [8]:
with open('./prediction_ts_format.pkl', 'wb') as f:
    pickle.dump(prediction_ts_format, f)

In [11]:
prediction_ts_format[15].data

feature                                target                    
representation                            0.1       0.5       0.9
offset          time_stamp                                       
0 days 01:00:00 2020-01-14 20:00:00 -0.001979 -0.004176  0.008872
                2020-01-14 21:00:00 -0.001996 -0.004266  0.008726
                2020-01-14 22:00:00 -0.002045 -0.004226  0.008949
                2020-01-14 23:00:00 -0.002136 -0.004017  0.009973
                2020-01-15 00:00:00 -0.002138 -0.004022  0.010138
...                                       ...       ...       ...
0 days 05:00:00 2020-08-28 23:00:00 -0.001773 -0.002339  0.011771
                2020-08-29 00:00:00 -0.001758 -0.002383  0.011627
                2020-08-29 01:00:00 -0.001748 -0.002412  0.011536
                2020-08-29 02:00:00 -0.001734 -0.002439  0.011420
                2020-08-29 03:00:00 -0.001722 -0.002462  0.011322

[27260 rows x 3 columns]